In [1]:
import os
from fig_gen  import FigureGenerator as fg
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
import seaborn as sns
import sklearn
from sklearn.linear_model import LinearRegression
%matplotlib inline
import missingno as msno
np.random.seed(19)
from matplotlib.pyplot import figure
data_directory = '/work2/05067/nagyz/austin_energy/data/'
filepaths = [os.path.join(data_directory,f) for f in os.listdir(data_directory) if f.endswith('.csv')]
from datetime import datetime 
from Graphs import load_data,load_graph_kwh,yearly_graph_kwh,load_graph_count,monthly_graph_kwh,daily_graph_kwh
from Graphs import load_graph_kwh_and_count,yearly_graph_kwh_and_count,monthly_graph_kwh_and_count,daily_graph_kwh_and_count
from agg_and_heatmap_functions import length,prav3,prav2,prav1,dinostorm,plot_aggraph,heatmap,finalheatmap
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, SGDRegressor
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor,GradientBoostingRegressor
from sklearn.utils import resample
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Ridge,Lasso
from sklearn.model_selection import GridSearchCV,KFold
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
weather_daily=pd.read_csv('Austin_weather_daily.csv')
weather_daily['time']=pd.to_datetime(weather_daily['time'],utc=False)
weather_daily1=weather_daily.set_index('time',drop=True)
weather_austin_daily=weather_daily1['tavg']
# NOT UTC TIME
weather_Pred=pd.read_csv('Austin_hourly.csv')
weather_Pred['time']=pd.to_datetime(weather_Pred['time'])
weather_pred1=weather_Pred.set_index('time',drop=True)
weather_austin_hourl_pred=weather_pred1['temp']
from sklearn.preprocessing import MinMaxScaler
import time
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from adtk.data import validate_series
from adtk.visualization import plot
from adtk.detector import SeasonalAD
from adtk.data import to_events
from adtk.detector import OutlierDetector
from scipy.stats import t
import torch
import time
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import random
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import copy
from sklearn.metrics import roc_auc_score,recall_score
from torch.utils.data import TensorDataset
from model_fns import clean_data,make_features,preprocess_data,prediction_model,check_outlier,split_sequences
from model_fns import SelfAttention,Encoder,Decoder,RecurrentAutoencoder,train_lstmae_model,val_lstmae_model,find_threshold,predict_anomaly

In [2]:
id_list = pd.read_csv('buildingInfo.csv')

#id_list.drop('Unnamed: 0',axis=1,inplace=True)
id_list['buildingType'].unique()
res_type = ['SINGLE FAMILY', 'MULTIFAMILY',  'FOURPLEX', 
            'DUPLEX', 'CONDOS', 'CONDO (STACKED)', 
            '1 FAM DWELLING, ACCESSORY DWELLING UNIT',
            'TOWNHOMES','MOHO SINGLE PP', 
            '1 FAM DWELLING, GARAGE APARTMENT', 'MOHO DOUBLE PP',
            'MOHO SINGLE REAL', 'MOHO DOUBLE REAL', 'TRIPLEX', 
            '1 FAM DWELLING, MOHO DOUBLE REAL',
           '1 FAM DWELLING, MOHO SINGLE REAL']
res_type_id_list = id_list[id_list['buildingType'].isin(res_type)].copy()
good_servicepoints_without_sampling=pd.read_csv('good_sp_without_sampling.csv')
good_servicepoints_with_sampling=pd.read_csv('good_sp_with_sampling.csv')

In [3]:
single_Famiy=res_type_id_list[res_type_id_list['buildingType']=='SINGLE FAMILY']
E_Res_singlefamily=single_Famiy[single_Famiy['rate']=='E-RES']
E_Res_singlefamily.head()

,Service Point,zipCode,rate,buildingType,meterCount,squareFt
4,-1608,78759,E-RES,SINGLE FAMILY,7996,17723356.0
31,-1581,78758,E-RES,SINGLE FAMILY,5363,8224307.5
66,-1546,78757,E-RES,SINGLE FAMILY,6174,8977316.0
84,-1528,78756,E-RES,SINGLE FAMILY,1727,2566446.0
102,-1510,78754,E-RES,SINGLE FAMILY,3752,5866178.5


In [4]:
recall_ensemble_scores = []
recall_baseline_scores = []
for i in tqdm(E_Res_singlefamily['Service Point']):
    servicepoint=-1581
    kmeans_lof_result_,df_lstm=check_outlier(servicepoint,num_kmeans_outlier=10)
    quantile_50=kmeans_lof_result_['Energy_kwh'].quantile(0.5)
    kmeans_lof_result_[f'{servicepoint}true_anomaly']=0
    kmeans_lof_result_[f'{servicepoint}true_anomaly'] = np.where((kmeans_lof_result_['tavg'] < 5) &
                                        (kmeans_lof_result_['Energy_kwh'] < quantile_50), 1,  kmeans_lof_result_[f'{servicepoint}true_anomaly'])
    kmeans_lof_result_[f'{servicepoint}true_anomaly'] = np.where((kmeans_lof_result_['tavg'] > 30) & 
                                        (kmeans_lof_result_['Energy_kwh'] < quantile_50), 1, kmeans_lof_result_[f'{servicepoint}true_anomaly'])
    length=12
    dataframe=df_lstm[['difference']]
    train_sequences,val_sequences,test_sequences=split_sequences(length,dataframe)
    rnn_model = RecurrentAutoencoder(seq_len=12,n_features=1, embedding_dim=64,num_layers=2,dropout_rate=0.2)
    criterion = nn.L1Loss(reduction='sum')
    optimizer = optim.Adam(rnn_model.parameters(), lr=0.001)
    total_epochs=50
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    best_val_loss = float('inf')
    patience = 10
    early_stop_counter = 0
    best_model = None  # initialize variable to hold the best model

    for epoch in range(total_epochs):
      start = time.time()
      train_loss = train_lstmae_model(train_sequences, rnn_model, epoch, total_epochs, optimizer, criterion)
      val_loss = val_lstmae_model(val_sequences, rnn_model, epoch, total_epochs, criterion)
      #print(f'Epoch {epoch}-, Training_loss : {train_loss} ,Validation_loss : {val_loss}, Time :{round(start-time.time(),2)}')
      if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0
        best_model = copy.deepcopy(rnn_model.state_dict())
      else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
          #print(f'Validation loss did not improve for {patience} epochs. Training stopped.')
          break
    print(best_val_loss)
    rnn_model.load_state_dict(best_model)
    th=find_threshold(rnn_model,val_sequences)
    lstm_results=predict_anomaly(rnn_model,test_sequences,th,df_lstm)
    agg_dict = {'error': 'sum', 'anomaly': lambda x: x.mode()[0]}
    final_lstm_results=lstm_results.resample('D').agg(agg_dict)
    final_lstm_results1=final_lstm_results.rename(columns={'anomaly':'lstm_anomaly'})
    final_results=final_lstm_results1.merge(kmeans_lof_result_,left_index=True,right_index=True)
    final_results[f'{servicepoint}majority_vote'] = final_results[['lstm_anomaly', 
                                                    f'{servicepoint}lof_is_outlier', f'{servicepoint}k_means_is_outlier']].mode(axis=1)[0]
    recall_ensemble = recall_score(final_results[[f'{servicepoint}true_anomaly']], final_results[[f'{servicepoint}majority_vote']], pos_label=1)
    recall_baseline = recall_score(final_results[[f'{servicepoint}true_anomaly']], final_results[[f'{servicepoint}k_means_is_outlier']], pos_label=1)
    recall_ensemble_scores.append(recall_ensemble)
    recall_baseline_scores.append(recall_baseline)

  0%|          | 0/46 [00:00<?, ?it/s]

0.9663


  2%|▏         | 1/46 [02:22<1:47:09, 142.89s/it]

0.9566


  4%|▍         | 2/46 [04:43<1:43:38, 141.34s/it]

...

 96%|█████████▌| 44/46 [1:34:21<04:13, 126.70s/it]

2.481


 98%|█████████▊| 45/46 [1:36:31<02:07, 127.48s/it]

1.1321


100%|██████████| 46/46 [1:38:40<00:00, 128.71s/it]


In [8]:
print(sum(recall_ensemble_scores)/len(recall_ensemble_scores))

0.8188405797101449


In [10]:
print(sum(recall_baseline_scores)/len(recall_baseline_scores))

0.6666666666666671


In [15]:
difference=sum(recall_ensemble_scores)/len(recall_ensemble_scores)-sum(recall_baseline_scores)/len(recall_baseline_scores)
difference/(sum(recall_baseline_scores)/len(recall_baseline_scores))*100

22.82608695652166

In [65]:
final_results

,error,lstm_anomaly,mean_-1581,-1581k_means_is_outlier,-1581lof_is_outlier,difference,Energy_kwh,prediction,tavg,-1581true_anomaly,-1581majority_vote
2021-02-11,2.148205,0,15.129611,0,0,20775.005427,137313.544434,116538.539007,0.8,0,0
2021-02-12,2.367068,0,17.376738,0,0,26541.061240,152739.028931,126197.967691,-0.6,0,0
2021-02-13,2.360956,0,17.322535,1,0,27648.633468,159610.781616,131962.148148,-1.6,0,0
2021-02-14,2.689737,0,19.707427,1,0,33350.374624,169227.443970,135877.069346,-4.3,0,0
2021-02-15,26.035945,1,-49.368068,1,1,-54928.963546,111264.153870,166193.117415,-9.0,1,1
2021-02-16,32.743857,1,-150.817251,1,1,-99116.246575,65719.435852,164835.682427,-7.6,1,1
2021-02-17,3.738546,0,-89.581584,0,1,-63953.996789,71391.902344,135345.899133,-1.8,1,0
2021-02-18,1.846328,0,-9.434697,1,0,-11504.061330,121933.559631,133437.620961,-1.8,0,0
2021-02-19,0.954144,0,1.276823,1,0,1701.532742,133263.058716,131561.525973,-0.5,0,0
2021-02-20,1.694146,0,9.720983,0,0,12029.607557,123748.882935,111719.275378,6.6,0,0


In [66]:
recall_ensemble = recall_score(final_results[[f'{servicepoint}true_anomaly']], final_results[[f'{servicepoint}majority_vote']], pos_label=1)
recall_baseline = recall_score(final_results[[f'{servicepoint}true_anomaly']], final_results[[f'{servicepoint}k_means_is_outlier']], pos_label=1)
print("recall_ensemble Score:", recall_ensemble)
print("recall_baseline Score:", recall_baseline)

recall_ensemble Score: 0.6666666666666666
recall_baseline Score: 0.6666666666666666
